In [3]:
def normalize_columns(df, inplace=False):
    """
    Normalise les noms de colonnes d'un DataFrame en snake_case sans accents ni caractères spéciaux.
    Règles :
      - retire les accents
      - met en minuscules
      - remplace tout caractère non alphanumérique par un underscore
      - réduit les underscores multiples en un seul
      - supprime les underscores en début/fin
      - si le nom commence par un chiffre, préfixe par 'c_'
      - si le résultat est vide, remplace par 'unknown'
      - garantit l'unicité des noms en ajoutant des suffixes _2, _3, ...
    Arguments :
      df : pandas.DataFrame
      inplace : bool (False par défaut). Si True, renomme les colonnes sur place et retourne le même objet.
    Retour :
      pandas.DataFrame avec colonnes normalisées.
    """
    import unicodedata
    import re
    import pandas as pd

    if not inplace:
        df = df.copy()

    def _slug(name: object) -> str:
        s = "" if name is None else str(name)
        # Normaliser unicode et séparer les accents
        s = unicodedata.normalize("NFKD", s)
        # Enlever les caractères de composition (accents)
        s = "".join(ch for ch in s if not unicodedata.combining(ch))
        s = s.lower()
        # Remplacer tout caractère non alphanumérique par underscore
        s = re.sub(r"[^a-z0-9]+", "_", s)
        # Réduire underscores multiples et trim
        s = re.sub(r"_+", "_", s).strip("_")
        # Préfixer si commence par chiffre
        if re.match(r"^[0-9]", s):
            s = "c_" + s
        if s == "":
            s = "unknown"
        return s

    # Appliquer la normalisation
    orig_cols = list(df.columns)
    normalized = [_slug(c) for c in orig_cols]

    # Garantir l'unicité des noms
    seen = {}
    unique_cols = []
    for name in normalized:
        base = name
        if name not in seen:
            seen[name] = 1
            unique_cols.append(name)
        else:
            seen[name] += 1
            new_name = f"{base}_{seen[name]}"
            # garantir que new_name lui-même n'existe pas déjà
            while new_name in seen:
                seen[base] += 1
                new_name = f"{base}_{seen[base]}"
            seen[new_name] = 1
            unique_cols.append(new_name)

    # Renommer le DataFrame
    mapping = dict(zip(orig_cols, unique_cols))
    df = df.rename(columns=mapping)

    return df